In [1]:
import pandas as pd

In [2]:
from numba import jit,cuda
@jit
def read_data():
    test_csv = pd.read_csv('datasets/cleaned_test_data.csv')
    train_csv = pd.read_csv('datasets/clean_train.csv')
    test_csv = test_csv.drop(test_csv.index[0])
    return train_csv,test_csv
train_csv,test_csv = read_data()

C:\Users\Jerry\AppData\Local\Temp\ipykernel_21512\3280737460.py:3: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def read_data():
C:\Users\Jerry\AppData\Local\Temp\ipykernel_21512\3280737460.py:2: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "read_data" failed type inference due to: Unknown attribute 'read_csv' of type Module(<module 'pandas' from 'c:\\Users\\Jerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\pandas\\__init__.py'>)

File "..\..\..\..\..\AppData\Local\Temp\ipykernel_21512\3280737460.py", line 4:
<source missing, REPL/exec in use?>

During: typing of get attribute at C:\Users\Jerry\Ap

In [4]:
# categories = {'Society & Culture' :1,'Science & Mathematics':2,'Health':3, 'Education & Reference':4,
#             'Computers & Internet' :5,'Sports' :6,'Business & Finance' :7,'Entertainment & Music' : 8,
#             'Family & Relationships':9, 'Politics & Government':10}
# train_csv['class']= train_csv['class'].map(categories)
train_csv

,text,class
0,nt optical mouse work glass table even surfac...,5
1,best offroad motorcycle trail longdistance tr...,6
2,trans fat reduce heard tras fat bad body f...,3
3,many planes fede heard largest airline world ...,7
4,san francisco bay area make sense rent buy ...,7
...,...,...
768320,believe hopelessness believe religion lack ...,1
768321,get horse s skeletal muscular systems web prin...,4
768322,quest promote racial equality government media...,10
768323,ways sell video games like want sell video ga...,7


In [3]:
df_balanced = train_csv[train_csv['class']==1].sample(30000)
for index in range(2,11):
    df_balanced = pd.concat([df_balanced,train_csv[train_csv['class']==index].sample(30000)])

In [4]:
x_train = list(df_balanced['text'])
y_train = df_balanced['class']
x_test = list(test_csv['text'])
y_test = test_csv['class']

In [5]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


from sklearn.linear_model import LogisticRegression
logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(multi_class='multinomial', solver='lbfgs')),
               ])
@jit
def trainModel():
    logreg.fit(x_train, y_train)
trainModel()



C:\Users\Jerry\AppData\Local\Temp\ipykernel_21512\3377814182.py:12: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def trainModel():
C:\Users\Jerry\AppData\Local\Temp\ipykernel_21512\3377814182.py:11: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "trainModel" failed type inference due to: Untyped global name 'logreg': Cannot determine Numba type of <class 'sklearn.pipeline.Pipeline'>

File "..\..\..\..\..\AppData\Local\Temp\ipykernel_21512\3377814182.py", line 13:
<source missing, REPL/exec in use?>

  @jit
c:\Users\Jerry\AppData\Local\Programs\Python\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: Numba

In [44]:
sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier()),
               ])
@jit
def svmModel():
    sgd.fit(x_train, y_train)
svmModel()

C:\Users\Jerry\AppData\Local\Temp\ipykernel_2988\942442530.py:6: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def svmModel():
C:\Users\Jerry\AppData\Local\Temp\ipykernel_2988\942442530.py:5: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "svmModel" failed type inference due to: Untyped global name 'sgd': Cannot determine Numba type of <class 'sklearn.pipeline.Pipeline'>

File "..\..\..\..\..\AppData\Local\Temp\ipykernel_2988\942442530.py", line 7:
<source missing, REPL/exec in use?>

  @jit
c:\Users\Jerry\AppData\Local\Programs\Python\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Functio

In [6]:
y_pred = logreg.predict(x_test)

In [8]:
from sklearn.metrics import accuracy_score, classification_report
print('accuracy %s' % accuracy_score(y_pred, y_test.to_numpy().astype(int)))
print(classification_report(y_test.to_numpy().astype(int), y_pred))


accuracy 0.7327780142275186
              precision    recall  f1-score   support

           1       0.65      0.59      0.62      3654
           2       0.70      0.78      0.74      2845
           3       0.80      0.81      0.80      3435
           4       0.59      0.55      0.57      3131
           5       0.81      0.89      0.85      3354
           6       0.86      0.87      0.87      2732
           7       0.61      0.56      0.58      2810
           8       0.69      0.73      0.71      3470
           9       0.77      0.77      0.77      3958
          10       0.81      0.77      0.79      3505

    accuracy                           0.73     32894
   macro avg       0.73      0.73      0.73     32894
weighted avg       0.73      0.73      0.73     32894



In [21]:
from sklearn.svm import LinearSVC
linear_svm =  Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LinearSVC()),
               ])
@jit
def linSVM():
    linear_svm.fit(x_train, y_train)
linSVM()

C:\Users\Jerry\AppData\Local\Temp\ipykernel_12592\1284080648.py:7: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def linSVM():
C:\Users\Jerry\AppData\Local\Temp\ipykernel_12592\1284080648.py:6: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "linSVM" failed type inference due to: Untyped global name 'linear_svm': Cannot determine Numba type of <class 'sklearn.pipeline.Pipeline'>

File "..\..\..\..\..\AppData\Local\Temp\ipykernel_12592\1284080648.py", line 8:
<source missing, REPL/exec in use?>

  @jit
c:\Users\Jerry\AppData\Local\Programs\Python\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning

In [24]:
lin_predictions=linear_svm.predict(x_test)
print(classification_report(y_test.to_numpy().astype(int), y_pred))

              precision    recall  f1-score   support

           1       0.67      0.59      0.63      3654
           2       0.72      0.78      0.75      2845
           3       0.80      0.82      0.81      3435
           4       0.59      0.56      0.58      3131
           5       0.80      0.89      0.85      3354
           6       0.87      0.88      0.87      2732
           7       0.61      0.57      0.59      2810
           8       0.70      0.73      0.72      3470
           9       0.78      0.78      0.78      3958
          10       0.81      0.78      0.80      3505

    accuracy                           0.74     32894
   macro avg       0.73      0.74      0.74     32894
weighted avg       0.74      0.74      0.74     32894



In [28]:
testing = {1: 'Society & Culture', 2: 'Science & Mathematics', 3: 'Health', 4: 'Education & Reference',
              5: 'Computers & Internet', 6: 'Sports', 7: 'Business & Finance', 8: 'Entertainment & Music',
              9: 'Family & Relationships', 10: 'Politics & Government'}
while(True):
    text = input('Enter a message')
    if(text == 'exit'):
        break
    else:
        print(testing[logreg.predict([text])[0]])

Entertainment & Music
Entertainment & Music
Business & Finance
Science & Mathematics
Politics & Government
Health
Entertainment & Music
Politics & Government
Politics & Government
Politics & Government
Politics & Government
Health
Entertainment & Music
Society & Culture
Entertainment & Music
Entertainment & Music
Education & Reference
Computers & Internet
Science & Mathematics
Computers & Internet
Science & Mathematics
Science & Mathematics
Science & Mathematics
Science & Mathematics


In [12]:
from sklearn.model_selection import GridSearchCV
import numpy as np
penalty = ['l1', 'l2']
C = np.logspace(-4,4,20)
hyperparameters = dict(penalty=penalty, C=C)
clf = GridSearchCV(LogisticRegression(), hyperparameters, cv=10)
logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', clf),
               ])
best_model = logreg.fit(x_train,y_train)


KeyboardInterrupt: 

In [27]:
import pickle
filename = 'logreg_model.sav'
pickle.dump(logreg, open(filename, 'wb'))